# QECC-733

Whether quantum error-correction code (QECC) `((7,3,3))` exists or not is an open question. This notebook is to explore the possibility of QECC-733.

In [ ]:
import os
os.environ['MKL_NUM_THREADS'] = '1'
os.environ['OMP_NUM_THREADS'] = '1'
import numpy as np
import torch

# more threads seem to slow things down
if torch.get_num_threads() != 1:
    torch.set_num_threads(1)

import numqi


In [ ]:
class VarQECSchmidt(torch.nn.Module):
    def __init__(self, num_qubit, num_logical_dim, error_op_full):
        super().__init__()
        self.num_logical_dim = num_logical_dim
        self.num_qubit = num_qubit
        self.error_op_torch = torch.tensor(np.stack(error_op_full, axis=0).reshape(-1,2**num_qubit)).to_sparse_csr()
        np_rng = np.random.default_rng()
        tmp0 = np_rng.normal(size=(2,num_logical_dim,2**num_qubit))
        self.theta = torch.nn.Parameter(torch.tensor(tmp0, dtype=torch.float64))
        self.q0_torch = None
        self.mask = torch.triu(torch.ones(num_logical_dim, num_logical_dim, dtype=torch.complex128), diagonal=1)

    def forward(self):
        q0 = torch.complex(self.theta[0], self.theta[1])
        q0_orth = []
        for ind0 in range(q0.shape[0]):
            tmp0 = q0[ind0]
            for x in q0_orth:
                tmp0 = tmp0 - torch.vdot(x,tmp0)*x
            q0_orth.append(tmp0 / torch.linalg.norm(tmp0))
        q0 = torch.stack(q0_orth, dim=1)
        inner_product = q0.T.conj() @ (self.error_op_torch @ q0).reshape(-1, *q0.shape)
        self.q0_torch = q0.detach().T
        tmp0 = (inner_product*self.mask).reshape(-1)
        tmp1 = torch.diagonal(inner_product, dim1=1, dim2=2).real
        tmp2 = (tmp1 - tmp1.mean(axis=1, keepdims=True)).reshape(-1)
        loss = torch.vdot(tmp0, tmp0).real + torch.dot(tmp2, tmp2)
        return loss


In [ ]:
num_qubit = 7
num_logical_dim = 3
distance = 3
error_op_full = numqi.qec.make_error_list(num_qubit, distance, tag_full=True)
model = VarQECSchmidt(num_qubit, num_logical_dim, error_op_full)
kwargs = dict(theta0=('uniform',-1,1), tol=1e-7, print_freq=0, early_stop_threshold=0.01)
theta_optim = numqi.optimize.minimize(model, num_repeat=2, **kwargs)


If you can minimize the loss function to almost `0`, then you have found a QECC-733.